<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Computação em Nuvem III
Caderno de **Exercícios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Introdução;</li>
  <li>Apache Spark;</li>
  <li>Data Wrangling com Spark.</li>
</ol>

---

# **Exercícios**

## 1\. Apache Spark

Replique as atividades do item 2.1 e 2.2 para instalar e configurar um cluster Apache Spark na máquina virtual do Google Colab.

* **Instalção**

Spark é uma aplicação desenvolvida na linguagem de programação Scala, que
funciona em uma máquina virtual Java (JVM). Por isso, é necessário fazer o download
da aplicação e instalar o Java em todas as máquinas (nós) do cluster.

Download do Spark, versão 3.0.0

In [1]:
%%capture
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-bin-hadoop2.7.tgz && rm spark-3.0.0-bin-hadoop2.7.tgz

Download e instalação do Java, versão 8.

In [2]:
%%capture
!apt-get remove openjdk*
!apt-get update --fix-missing
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

> **Nota:** a versão do PySpark deve ser a mesma que a versão da aplicação Spark.

In [3]:
!pip install -q pyspark==3.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.7/204.7 MB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 14.9 MB/s eta 0:00:00


* **Configuração**

Na etapa de configuração, é necessário configurar as máquinas (nós) do cluster para
que tanto a aplicação do Spark quanto a instalação do Java possam ser encontrados
pelo PySpark e, consequentemente, pelo Python. Para isso, basta preencher as variáveis
de ambiente `JAVA_HOME` e `SPARK_HOME` com o seus respectivos caminhos de
instalação

In [4]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"

Por fim, para conectar o PySpark (e o Python) ao Spark e ao Java, pode-se utilizar o
pacote Python.

In [5]:
!pip install -q findspark==1.4.2

O método `init()` injeta as variáveis de ambiente `JAVA_HOME` e `SPARK_HOME` no
ambiente de execução Python, permitindo assim a correta conexão entre o pacote
PySpark com a aplicação Spark.

In [6]:
import findspark

findspark.init()

## 2\. Data Wrangling

A base de dados presente neste [link](https://www.kaggle.com/datasets/bank-of-england/a-millennium-of-macroeconomic-data) contem dados macroeconômicos sobre o Reino Unido desde o século 13.

**2.1\. Data**

Faça o download dos dados utilizando a máquina virutal do Google Colab com o código abaixo.

In [7]:
!wget -q "https://raw.githubusercontent.com/cluster-apps-on-docker/spark-standalone-cluster-on-docker/master/build/workspace/data/uk-macroeconomic-data.csv" -O "uk-macroeconomic-data.csv"

**2.2. Wrangling**

Processe os dados para que a base de dados final apresente os valores da taxa de desemprego (`Unemployment rate`) e população (`Population (GB+NI)`) estejam ordenados por ano decrescente:

```csv
year,population,unemployment_rate
...,...,...
```

Para isso, utilize:

 - Pandas

In [32]:
import pandas as pd

# DataFrame
df = pd.read_csv('uk-macroeconomic-data.csv')

# select das colunas
df = df[['Description', 'Population (GB+NI)', 'Unemployment rate']]


# renomeando as colunas
df.rename(columns= {'Description':'Year','Population (GB+NI)':'Population','Unemployment rate':'Unemployment'}, inplace=True)

pandas_data = df.dropna().drop(0).sort_values(by='Year', ascending=False, ignore_index=True)

pandas_data.head(10)

,Year,Population,Unemployment
0,2016,65573,4.90
1,2015,65110,5.38
2,2014,64597,6.18
3,2013,64106,7.61
4,2012,63705,7.97
5,2011,63285,8.11
6,2010,62759,7.87
7,2009,62260,7.61
8,2008,61824,5.69
9,2007,61319,5.33


 - PySpark

In [20]:
from pyspark.sql import SparkSession

# SparkSession
spark = SparkSession.builder.appName('UKMacroData').getOrCreate()

# DataFrame
data = spark.read.csv(path='uk-macroeconomic-data.csv', sep=',', header=True)

# select das colunas
data = data.select(['Description','Population (GB+NI)','Unemployment rate'])

# renomeando as colunas
data = (
    data
    .withColumnRenamed('Description', 'Year')
    .withColumnRenamed('Population (GB+NI)', 'Population')
    .withColumnRenamed('Unemployment rate', 'Unemployment')
)

# limpando e ordenando os dados por ano
pyspark_data = data.dropna().filter(data['Year'] != 'Units').orderBy(data['Year'].desc())

# visualizar dados
pyspark_data.show(10)

+----+----------+------------+
|Year|Population|Unemployment|
+----+----------+------------+
|2016|     65573|        4.90|
|2015|     65110|        5.38|
|2014|     64597|        6.18|
|2013|     64106|        7.61|
|2012|     63705|        7.97|
|2011|     63285|        8.11|
|2010|     62759|        7.87|
|2009|     62260|        7.61|
|2008|     61824|        5.69|
|2007|     61319|        5.33|
+----+----------+------------+
only showing top 10 rows

